# Association Rule Learning with Apriori Algorithm

## Case Study: Online Retail II

**Business Problem: Suggesting products to users when they are buying**
<br>
<br>



__Variables__

Invoice: Unique number for each invoice. If invoice number starts with 'C', it's mean this invoice cancelled.

StockCode: Unique number for each product.

Description: Prooduct name.

Quantity: Indicates how many units of the product were sold.

InvoiceDate: Invoice date.

Price: Product price for each units. (Sterling)

CustumerID: Unique ID for each customer.

Country: Customer country.

___

**Import Libraries**

In [108]:
import numpy as np
import pandas as pd
import warnings
from mlxtend.frequent_patterns import apriori, association_rules

warnings.filterwarnings('ignore')

**Read excel file and assign called df**

In [2]:
df_ = pd.read_excel('online_retail_II.xlsx', sheet_name='Year 2010-2011')

In [66]:
df = df_.copy()

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


**Check the df shape**

In [5]:
df.shape

(541910, 8)

**Check the missing value**

In [6]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

**Get th describe numerical values**

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0,9.552234,218.080957,-80995.00,1.00,3.00,10.00,80995.0
Price,541910.0,4.611138,96.759765,-11062.06,1.25,2.08,4.13,38970.0
Customer ID,406830.0,15287.684160,1713.603074,12346.00,13953.00,15152.00,16791.00,18287.0


## Data Preprocessing

**Outlier threshold**

In [8]:
def outlier_thresholds(dataframe, variable):
    
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    low_limit = quartile1 - 1.5*interquantile_range
    up_limit = quartile3 + 1.5*interquantile_range    
    
    return low_limit, up_limit

In [9]:
def replace_with_thresholds(dataframe, variable):
    
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[dataframe[variable] < low_limit, variable] = low_limit
    dataframe.loc[dataframe[variable] > up_limit, variable] = up_limit

In [67]:
def retail_data_prep(dataframe):
    
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe['Invoice'].str.contains('C', na=False)]
    dataframe = dataframe[dataframe['Quantity'] > 0]
    dataframe = dataframe[dataframe['Price'] > 0]
    replace_with_thresholds(dataframe, 'Quantity')
    replace_with_thresholds(dataframe, 'Price')
    
    return dataframe

In [68]:
df = retail_data_prep(df)

In [60]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0,11.830770,25.523052,1.000,2.00,6.00,12.00,298.50
Price,397885.0,2.893492,3.227175,0.001,1.25,1.95,3.75,37.06
Customer ID,397885.0,15294.416882,1713.144421,12346.000,13969.00,15159.00,16795.00,18287.00


**Preparing ARL data structure for France**

In [69]:
df_fr = df[df['Country'] == 'France']
df_fr

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24.0,2010-12-01 08:45:00,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24.0,2010-12-01 08:45:00,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12.0,2010-12-01 08:45:00,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12.0,2010-12-01 08:45:00,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24.0,2010-12-01 08:45:00,0.65,12583.0,France
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6.0,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4.0,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4.0,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3.0,2011-12-09 12:50:00,4.95,12680.0,France


In [15]:
df_fr.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24.0,2010-12-01 08:45:00,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24.0,2010-12-01 08:45:00,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12.0,2010-12-01 08:45:00,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12.0,2010-12-01 08:45:00,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24.0,2010-12-01 08:45:00,0.65,12583.0,France


In [16]:
df_fr.groupby(['Invoice', 'Description']).agg({'Quantity': 'sum'})

Quantity
Invoice Description                               
536370   SET 2 TEA TOWELS I LOVE LONDON       24.0
        ALARM CLOCK BAKELIKE GREEN            12.0
        ALARM CLOCK BAKELIKE PINK             24.0
        ALARM CLOCK BAKELIKE RED              24.0
        CHARLOTTE BAG DOLLY GIRL DESIGN       20.0
...                                            ...
581587  PACK OF 20 SPACEBOY NAPKINS           12.0
        PLASTERS IN TIN CIRCUS PARADE         12.0
        PLASTERS IN TIN STRONGMAN             12.0
        POSTAGE                                1.0
        SPACEBOY LUNCH BOX                    12.0

[8306 rows x 1 columns]

In [17]:
df_fr.groupby(['Invoice', 'Description']).agg({'Quantity': 'sum'})['Quantity'].unstack()

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
Invoice,,,,,,,,,,,,,,,,,,,,,
536370,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_fr.groupby(['Invoice', 'Description']).agg({'Quantity': 'sum'})['Quantity'].unstack().fillna(0)

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
Invoice,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_fr.groupby(['Invoice', 'Description']).agg({'Quantity': 'sum'})['Quantity'].unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)


Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
Invoice,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
def create_arl_data_structure(dataframe, id_=False):
    
    if id:
    
        dataframe = dataframe.groupby(['Invoice', 'StockCode'])['Quantity'].sum() \
        .unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
    
        return dataframe
    
    else:
        
        dataframe = dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum() \
        .unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
        
        return dataframe

In [57]:
create_arl_data_structure(df_fr, id_=False)

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP SUKI AND FRIENDS,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
Invoice,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C579532,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C579562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C580161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
fr_inv_pro_df = create_arl_data_structure(df_fr, id=True)
fr_inv_pro_df

StockCode,10002,10120,10125,10135,11001,15036,15039,16012,16048,16218,...,85232D,90030B,90030C,90184B,90184C,90201B,90201C,C2,M,POST
Invoice,,,,,,,,,,,,,,,,,,,,,
536370,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
581001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
581171,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
def check_describtion(dataframe, stockcode):
    
    description = dataframe[dataframe['StockCode'] == stockcode]['Description'].values[0]
    return description

In [33]:
check_describtion(df_fr, 10120)

'DOGGY RUBBER'

## Association Rule Analysis

In [110]:
frequent_itemsets = apriori(fr_inv_pro_df, min_support=0.01, use_colnames=True)

In [109]:
frequent_itemsets

,support,itemsets
0,0.020566,(10002)
1,0.015424,(10125)
2,0.010283,(16236)
3,0.012853,(16237)
4,0.012853,(16238)
...,...,...
40650,0.010283,"(22659, 23206, 22726, 22727, 22728, 20750, 223..."
40651,0.010283,"(22659, 23206, 22726, 22727, 22728, 20750, 223..."
40652,0.010283,"(22659, 23206, 22726, 22727, 22728, 20750, 223..."
40653,0.010283,"(22659, 23206, 22726, 22727, 22728, 22352, 232..."


In [34]:
frequent_itemsets.sort_values('support', ascending=False)

,support,itemsets
538,0.773779,(POST)
387,0.187661,(23084)
107,0.179949,(21731)
243,0.172237,(22554)
245,0.169666,(22556)
...,...,...
18793,0.010283,"(22729, 21086, 22326, 22551)"
18787,0.010283,"(23256, 21086, 22492, 22326)"
18786,0.010283,"(22728, 21086, 22492, 22326)"
18785,0.010283,"(21086, 22492, 22326, 22727)"


In [35]:
rules = association_rules(frequent_itemsets, metric='support', min_threshold=0.01)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(10002),(21791),0.020566,0.028278,0.010283,0.500000,17.681818,0.009701,1.943445
1,(21791),(10002),0.028278,0.020566,0.010283,0.363636,17.681818,0.009701,1.539111
2,(10002),(21915),0.020566,0.069409,0.010283,0.500000,7.203704,0.008855,1.861183
3,(21915),(10002),0.069409,0.020566,0.010283,0.148148,7.203704,0.008855,1.149771
4,(10002),(22551),0.020566,0.136247,0.010283,0.500000,3.669811,0.007481,1.727506
...,...,...,...,...,...,...,...,...,...
1372699,(23254),"(22659, 23206, 22726, 22727, 22728, 20750, 223...",0.071979,0.010283,0.010283,0.142857,13.892857,0.009543,1.154670
1372700,(22326),"(22659, 23206, 22726, 22727, 22728, 20750, 223...",0.159383,0.010283,0.010283,0.064516,6.274194,0.008644,1.057974
1372701,(21558),"(22659, 23206, 22726, 22727, 22728, 20750, 223...",0.051414,0.010283,0.010283,0.200000,19.450000,0.009754,1.237147
1372702,(23291),"(22659, 23206, 22726, 22727, 22728, 20750, 223...",0.041131,0.010283,0.010283,0.250000,24.312500,0.009860,1.319623


In [36]:
rules[(rules['support'] > 0.1) & (rules['confidence'] > 0.1) & (rules['lift'] > 5)].sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23706,"(21080, 21086)",(21094),0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612
23707,"(21080, 21094)",(21086),0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301
1777,(21094),(21086),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563
26078,"(POST, 21094)",(21086),0.107969,0.138817,0.102828,0.952381,6.860670,0.087840,18.084833
1776,(21086),(21094),0.138817,0.128535,0.123393,0.888889,6.915556,0.105550,7.843188
26076,"(POST, 21086)",(21094),0.118252,0.128535,0.102828,0.869565,6.765217,0.087628,6.681234
23708,"(21094, 21086)",(21080),0.123393,0.133676,0.100257,0.812500,6.078125,0.083762,4.620394
1609,(21094),(21080),0.128535,0.133676,0.102828,0.800000,5.984615,0.085646,4.331620
26081,(21094),"(POST, 21086)",0.128535,0.118252,0.102828,0.800000,6.765217,0.087628,4.408740
23711,(21094),"(21080, 21086)",0.128535,0.102828,0.100257,0.780000,7.585500,0.087040,4.078056


## Functionalize all steps

In [114]:
df = df_.copy()

In [115]:
def create_rules(dataframe, id=True, country='France'):
    
    dataframe = retail_data_prep(dataframe)
    dataframe = dataframe[dataframe['Country'] == country]
    dataframe = create_arl_data_structure(dataframe, id_=id)
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric='support', min_threshold=0.01)
    
    return rules   

In [116]:
rules = create_rules(df, id=True, country='France')

In [117]:
rules[(rules['support'] > 0.1) & (rules['confidence'] > 0.1) & (rules['lift'] > 5)].sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
23706,"(21080, 21086)",(21094),0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612
23707,"(21080, 21094)",(21086),0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301
1777,(21094),(21086),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563
26078,"(POST, 21094)",(21086),0.107969,0.138817,0.102828,0.952381,6.860670,0.087840,18.084833
1776,(21086),(21094),0.138817,0.128535,0.123393,0.888889,6.915556,0.105550,7.843188
26076,"(POST, 21086)",(21094),0.118252,0.128535,0.102828,0.869565,6.765217,0.087628,6.681234
23708,"(21094, 21086)",(21080),0.123393,0.133676,0.100257,0.812500,6.078125,0.083762,4.620394
1609,(21094),(21080),0.128535,0.133676,0.102828,0.800000,5.984615,0.085646,4.331620
26081,(21094),"(POST, 21086)",0.128535,0.118252,0.102828,0.800000,6.765217,0.087628,4.408740
23711,(21094),"(21080, 21086)",0.128535,0.102828,0.100257,0.780000,7.585500,0.087040,4.078056


## Product Recommendation

In [121]:
product_id = 22492
check_describtion(df, product_id)

'MINI PAINT SET VINTAGE '

In [120]:
sorted_rules = rules.sort_values('lift', ascending=False)
sorted_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1082358,"(22659, 23206, 22726, 22727)","(21558, 23254, 22326, 23199)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1160023,"(22727, 22728, 22352, 23254, 23199)","(22326, 21558, 22726, 20750)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1160025,"(22727, 22728, 21558, 23254, 22326)","(22352, 23199, 22726, 20750)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1160026,"(22727, 22728, 23254, 22326, 23199)","(22352, 21558, 22726, 20750)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1160027,"(22727, 22728, 21558, 22326, 23199)","(22352, 23254, 22726, 20750)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf


In [130]:
recommendation_list = []

for i, product in enumerate(sorted_rules['antecedents']):
    for j in product:
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]['consequents']))

In [168]:
def arl_recommender(rules_df, product_id, rec_count=3):
    recommendation_list = []

    for i, product in enumerate(sorted_rules['antecedents']):
        for j in product:
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]['consequents'])[0])
#     print([f'{x}: ' + check_describtion(df, x) for x in recommendation_list[0:rec_count]])
    return recommendation_list[0:rec_count]


In [169]:
arl_recommender(sorted_rules, product_id = 22492, rec_count=3)

['22556: PLASTERS IN TIN CIRCUS PARADE ', '22551: PLASTERS IN TIN SPACEBOY', '22326: ROUND SNACK BOXES SET OF4 WOODLAND ']


[22556, 22551, 22326]